In [ ]:
from isimip_client.client import ISIMIPClient
client = ISIMIPClient()

In [ ]:
# mask a file for a country
# run this subsequently to poll the status
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily_2015_2020.nc'
response = client.mask(path, country='deu')
response

In [ ]:
# when it it done get the url to download the result
client.download(response['file_url'])

In [ ]:
# the same works for a dataset, it just takes longer
path = 'ISIMIP3b/InputData/climate/atmosphere/bias-adjusted/global/daily/ssp370/GFDL-ESM4/gfdl-esm4_r1i1p1f1_w5e5_ssp370_pr_global_daily'
response = client.mask(path, country='deu')
response

In [ ]:
# when it it done get the url to download the result
client.download(response['file_url'])